# Download Subdirectories From FTP II

#### Updated: Dec 17, 2022

#  

Run for continuous downloading of subdirectories, concurrent with processing of subdirectories.

In [1]:
import pandas as pd
import os
import shutil
import time
from datetime import datetime
import ftplib
from random import randrange

Set parameters:

In [2]:
user = 'Rav'
process_on_VDI = True
wait = 4 #in minutes
stop_loop_threshold = 2640 #max while loops to prevent infinite loop

Set directories:

In [3]:
rootDir_local = 'C:/Users/rnaidoo/Documents/Projects_data/Alouette_I/' #files on C:/ are not persistent on VDI
downloadingDir = rootDir_local + '01_downloading/'
downloadedDir = rootDir_local + '02_downloaded/'
if process_on_VDI:
    logDir = 'L:/DATA/Alouette_I/06_log/' #DO NOT CHANGE
else:
    logDir = rootDir_local + '06_log/'

#  

#### Functions:

In [4]:
def move_images(old_dir, new_dir, roll, subdir, copy_to_other_drive=False):
    oldDir = old_dir + roll + '/' + subdir + '/'
    newDir = new_dir + roll + '/' + subdir + '/'
    os.makedirs(newDir, exist_ok=True)
    
    if copy_to_other_drive:
        for file in os.listdir(oldDir):
            shutil.copyfile(oldDir+file, newDir+file)
    else:
        for file in os.listdir(oldDir):
            os.rename(oldDir+file, newDir+file)
    
    shutil.rmtree(old_dir + roll + '/' + subdir + '/')
    if len(os.listdir(old_dir + roll + '/')) == 0:
        shutil.rmtree(old_dir + roll + '/')

In [5]:
def draw_random_subdir(roll, subdir_list, logDir):
    
    subdirectory = subdir_list[randrange(len(subdir_list))]
    
    #Check randomly-selected roll and subdirectory against the 'download_log'
    if os.path.exists(logDir + 'download_log.csv'):
        df_log = pd.read_csv(logDir + 'download_log.csv')
        df_search = df_log.loc[(df_log['Roll'] == roll) & (df_log['Subdirectory'] == subdirectory)]
        if len(df_search) > 0:
            print(roll + '/' + subdirectory + ' already downloaded!')
            return ''
        else:
            return subdirectory
    else:
        return subdirectory

In [6]:
def draw_random_subdir2(subdir_ids_list, logDir):
    
    subdir_id = subdir_ids_list[randrange(len(subdir_ids_list))]
    subdir_id_parts = subdir_id.split('_')
    roll = subdir_id_parts[0]
    subdirectory = subdir_id_parts[1]
    
    #Check randomly-selected roll and subdirectory against the 'download_log'
    if os.path.exists(logDir + 'download_log.csv'):
        df_log = pd.read_csv(logDir + 'download_log.csv')
        df_search = df_log.loc[(df_log['Roll'] == roll) & (df_log['Subdirectory'] == subdirectory)]
        if len(df_search) > 0:
            print(roll + '/' + subdirectory + ' already downloaded!')
            return ''
        else:
            return roll, subdirectory
    else:
        return roll, subdirectory
    

#  

#### Check if subdirectory needs to be downloaded, then download random subdirectory:

In [ ]:
stop_condition = False
stop_condition_counter = 0

while stop_condition == False:
    #Download a random subdirectory if '02_downloaded' is empty
    if len(os.listdir(downloadedDir)) == 0:
        # Connect to FTP Server
        HOSTNAME = "donnees-data.asc-csa.gc.ca"
        USERNAME = "Anonymous"
        PASSWORD = ""
        ftp = ftplib.FTP(HOSTNAME, USERNAME, PASSWORD)
        print('Connected to ftp server: ' + HOSTNAME)
        ftp_rootpath = '/users/OpenData_DonneesOuvertes/pub/AlouetteData/Alouette Data/'

        #Randomly draw roll and subdirectory (using draw_random_subdir2())
        df_inventory = pd.read_csv(logDir + 'image_inventory.csv')
        subdir_ids_tot = df_inventory['subdir_id'].unique()
        if os.path.exists(logDir + 'download_log.csv'):
            df_log = pd.read_csv(logDir + 'download_log.csv')
            subdir_ids_dl = df_log['subdir_id'].unique()
        else:
            subdir_ids_dl = []
        subdir_ids_rem = list(set(subdir_ids_tot) - set(subdir_ids_dl))
        roll, subdirectory = draw_random_subdir2(subdir_ids_list=subdir_ids_rem, logDir=logDir)

        #Set directories
        saveDir = downloadingDir + roll + '/' + subdirectory + '/'
        os.makedirs(saveDir, exist_ok=True)
        ftp.cwd(ftp_rootpath + '/' + roll + '/' + subdirectory + '/')

        start = time.time()
        n_dl = len(ftp.nlst())
        print('')
        print('Downloading ' + roll + '/' + subdirectory + '/ subdirectory ('  + str(n_dl) + ' images, ' + str(len(subdir_ids_rem)-1) + ' subdirectories to go)')
        for file in ftp.nlst():
            local_file = open(saveDir + file, "wb")
            ftp.retrbinary("RETR " + file, local_file.write)
            local_file.close()
            #print('Downloaded: ' + file)
        end = time.time()
        t = end - start
        print('Download time for subdirectory: ' + str(round(t/60, 1)) + ' min')
        print('')

        #Record subdirectory name in download_log
        df_result = pd.DataFrame({
            'Roll': roll,
            'Subdirectory': subdirectory,
            'Images_downloaded': n_dl,
            'Download_time': t,
            'Download_timestamp': datetime.fromtimestamp(end),
            'User': user,
            'subdir_id': roll + '_' + subdirectory
        }, index=[0])
        if os.path.exists(logDir + 'download_log.csv'):
            df_log = pd.read_csv(logDir + 'download_log.csv')
            df_update = pd.concat([df_log, df_result], axis=0, ignore_index=True)
            df_update.to_csv(logDir + 'download_log.csv', index=False)
        else:
            df_result.to_csv(logDir + 'download_log.csv', index=False)

        #Backup 'download_log' (10% of the time)
        if randrange(10) == 7:
            df_log = pd.read_csv(logDir + 'download_log.csv')
            datetime_str = datetime.now().strftime("%Y%m%d_%Hh%M")
            os.makedirs(logDir + 'backups/', exist_ok=True)
            df_log.to_csv(logDir + 'backups/' + 'download_log-' + datetime_str + '.csv', index=False)

        #Move fully downloaded subdirectory to '02_processing' folder
        move_images(old_dir=downloadingDir, new_dir=downloadedDir, roll=roll, subdir=subdirectory)
        
        stop_condition_counter += 1
    
    else:
        #Wait
        print('Wait ' + str(wait) + ' min')
        subdir_ids_rem = []
        time.sleep(wait*60)
    
    
    #Check stop conditions
    if len(subdir_ids_rem) == 1:
        print('Stop!')
        stop_condition = True
    if stop_condition_counter == stop_loop_threshold:
        print('Stop!')
        stop_condition = True
        

Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.2 min

Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.3 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.6 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.3 min

Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.6 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.2 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Connected to ftp server: don

Download time for subdirectory: 1.5 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.3 min

Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.6 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.2 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.1 min

Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.1 min

Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.2 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.0 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Downlo

Download time for subdirectory: 1.4 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.5 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.1 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.1 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.5 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.5 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subd

Download time for subdirectory: 1.6 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.6 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.7 min

Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.5 min

Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.8 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.5 min

Wait 4 min
Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.4 min

Wait 4 min
Connected to ftp server: donnees-data.asc-csa.gc.ca

Download time for subdirectory: 1.0 min

Wait 4 min
Wait 4 mi